In [11]:
import pandas as pd
import numpy as np

In [12]:
house = pd.read_csv("house.csv")
item = pd.read_csv("item.csv")
house_item = pd.read_csv("hi_interaction.csv")
house_card = pd.read_csv("hc_interaction.csv")
card = pd.read_csv("card.csv")

# house와 card 1:1 매칭된 새로운 house 생성

In [13]:
house = house.rename(columns={"공간":"space", "평수":"size", "작업":"work", "분야":"category", "가족형태":"family", "지역":"region", "스타일":"style", "기간":"duration","예산":"budget","세부공사":"detail","좋아요":"prefer","스크랩":"scrab","댓글":"comment","조회":"views"})

In [14]:
hc_temp = {i:j for i, j in zip(house_card.card.values, house_card.house.values)}
card['house_id'] = card.card.map(hc_temp)

house.views = house.views.apply(lambda x: x.replace(',', ''))
house.scrab = house.scrab.apply(lambda x: x.replace(',', ''))
house.prefer = house.prefer.apply(lambda x: x.replace(',', ''))

house.views = pd.to_numeric(house.views)
house.scrab = pd.to_numeric(house.scrab)
house.prefer = pd.to_numeric(house.prefer)

# item.csv 전처리

In [15]:
item = item.fillna(np.nan)

## predict_price 예상가격만 남기기

In [16]:
predict_price_list = []

import re
p = re.compile('.+(?=사)')
for i in item.predict_price.index:
    m = p.findall(str(item.predict_price.iloc[i]))
    if len(m) == 0:
        m.append("예상가정보없음")
    predict_price_list.append(m[0][3:])

In [17]:
item.predict_price = predict_price_list

## 별점 숫자만 남기기

In [18]:
item.rating = item.rating.apply(lambda x: str(x)[3:-1])

In [19]:
item.rating = item.rating.apply(lambda x: np.nan if x == '' else x)

## review 리뷰 개수만 남기기

In [20]:
item.review = item.review.fillna("리뷰 쓰기첫 리뷰 두 배 적립")

In [21]:
review_list = []

import re
p = re.compile('[(][0-9]*[)]')
for idx, review in enumerate(item.review):
    m = p.findall(review)
    if len(m) == 0:
        m.append("(0)")
    review_list.append(m[0][1:-1])

item.review = review_list

## 카테고리 결측값 기타로, 쌍따옴표 들어가는 경우 제거

In [22]:
item.fillna("기타", inplace=True)

In [23]:
item.category = item.category.apply(lambda x: x.split('|')[0].strip('"'))

# house.csv

In [24]:
house["size"] = house["size"].fillna("0평").apply(lambda x:int(x.rstrip("평대이상하미만 ")))

# csv 파일 추출

In [26]:
item.to_csv("item.tsv", sep="\t", index=False)
house.to_csv("house.tsv", sep="\t", index=False)
card.to_csv("card.tsv", sep="\t", index=False)